In [2]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [16]:
from pathlib import Path

from wikipedia_cleanup.data_filter import KeepAttributesDataFilter, generate_default_filters
from wikipedia_cleanup.predict import TrainAndPredictFramework
from wikipedia_cleanup.random_forest import RandomForestPredictor

import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [17]:
model = RandomForestPredictor(use_cache=False)
framework = TrainAndPredictFramework(model, group_key=['infobox_key', 'property_name'],test_start_date=datetime(2017, 9, 1))

In [70]:
framework.data = pd.read_csv('../../data/popular_data_with_features.csv')[:100000]

In [71]:
from datetime import datetime
df_counts = framework.data.groupby(['infobox_key', 'property_name'],sort=False).count()["value_valid_from"]

In [73]:
lst=[]
start=0
for key,count in tqdm(df_counts.iteritems()):
    if count>=200:
        lst.append(framework.data[start:start+count])
    start+=count
data_df_small=pd.concat(lst)
data_df_small.shape

11841it [00:00, 1975448.62it/s]


(2130, 20)

In [74]:
#number of keys
data_df_small.groupby(['infobox_key', 'property_name'],sort=False).count()["value_valid_from"].shape

(8,)

In [75]:
framework.data = data_df_small

In [76]:
framework.data

,infobox_key,property_name,template,value_valid_from,day_of_year,day_of_month,day_of_week,month_of_year,quarter_of_year,is_month_start,is_month_end,is_quarter_start,is_quarter_end,days_since_last_change,days_since_last_2_changes,days_since_last_3_changes,days_until_next_change,days_between_last_and_2nd_to_last_change,mean_change_frequency_all_previous,mean_change_frequency_last_3
7621,60103127-0,genre,infobox_musical_artist,2007-01-03,3,3,2,1,1,False,False,False,False,0,0,0,29,0,0.000000,0.000000
7622,60103127-0,genre,infobox_musical_artist,2007-02-01,32,1,3,2,1,True,False,False,False,29,0,0,1,0,29.000000,0.000000
7623,60103127-0,genre,infobox_musical_artist,2007-02-02,33,2,4,2,1,False,False,False,False,1,30,0,2,29,15.000000,0.000000
7624,60103127-0,genre,infobox_musical_artist,2007-02-04,35,4,6,2,1,False,False,False,False,2,3,32,7,1,10.666667,10.666667
7625,60103127-0,genre,infobox_musical_artist,2007-02-11,42,11,6,2,1,False,False,False,False,7,9,10,4,2,9.750000,3.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75081,382716487-0,num_episodes,infobox_television,2019-08-07,219,7,2,8,3,False,False,False,False,46,53,60,3,7,14.953488,20.000000
75082,382716487-0,num_episodes,infobox_television,2019-08-10,222,10,5,8,3,False,False,False,False,3,49,56,7,46,14.898148,18.666667
75083,382716487-0,num_episodes,infobox_television,2019-08-17,229,17,5,8,3,False,False,False,False,7,10,56,7,3,14.861751,18.666667
75084,382716487-0,num_episodes,infobox_television,2019-08-24,236,24,5,8,3,False,False,False,False,7,14,17,7,7,14.825688,5.666667


In [97]:
framework.data["value_valid_from"] = pd.to_datetime(framework.data["value_valid_from"]).dt.tz_localize(None)

/opt/homebrew/Caskroom/miniforge/base/envs/wikipedia/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [78]:
group_key=['infobox_key', 'property_name']
framework.data["key"] = list(
            zip(*[framework.data[group_key] for group_key in framework.group_key])
        )

In [82]:
framework.data = framework.data[framework.data['infobox_key']=='382716487-0']

In [98]:
framework.fit_model()

Start training.


100%|██████████| 1/1 [00:00<00:00,  9.57it/s]

Finished training. Time elapsed: 0:00:00.108601


In [95]:
framework.data[(framework.data['value_valid_from']<pd.to_datetime('2017-09-15')) & (framework.data['value_valid_from']>=pd.to_datetime('2017-08-25'))]

,infobox_key,property_name,template,value_valid_from,day_of_year,day_of_month,day_of_week,month_of_year,quarter_of_year,is_month_start,...,is_quarter_start,is_quarter_end,days_since_last_change,days_since_last_2_changes,days_since_last_3_changes,days_until_next_change,days_between_last_and_2nd_to_last_change,mean_change_frequency_all_previous,mean_change_frequency_last_3,key
75034,382716487-0,num_episodes,infobox_television,2017-08-26,238,26,5,8,3,False,...,False,False,7,14,21,7,7,14.904762,7.0,"(382716487-0, num_episodes)"
75035,382716487-0,num_episodes,infobox_television,2017-09-02,245,2,5,9,3,False,...,False,False,7,14,21,10,7,14.857988,7.0,"(382716487-0, num_episodes)"
75036,382716487-0,num_episodes,infobox_television,2017-09-12,255,12,1,9,3,False,...,False,False,10,17,24,4,7,14.829412,8.0,"(382716487-0, num_episodes)"


In [100]:
framework.test_model(randomize=False, predict_subset=1, save_results=True)

  0%|          | 0/1 [00:00<?, ?it/s]

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  19  20  21  23
  27  39  45  49  53  63  71  84  97 100 106 119 175 184 187]
[0.   0.   0.   0.02 0.   0.   0.85 0.   0.   0.   0.   0.   0.   0.02
 0.   0.   0.02 0.   0.   0.   0.   0.07 0.   0.01 0.   0.   0.   0.
 0.   0.01 0.   0.   0.  ]
0.0
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  19  20  21  23
  27  39  45  49  53  63  71  84  97 100 106 119 175 184 187]
[0.   0.   0.   0.02 0.   0.   0.85 0.   0.   0.   0.   0.   0.   0.02
 0.   0.   0.02 0.   0.   0.   0.   0.07 0.   0.01 0.   0.   0.   0.
 0.   0.01 0.   0.   0.  ]
0.85
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  19  20  21  23
  27  39  45  49  53  63  71  84  97 100 106 119 175 184 187]
[0.   0.   0.   0.02 0.   0.   0.85 0.   0.   0.   0.   0.   0.   0.02
 0.   0.   0.02 0.   0.   0.   0.   0.07 0.   0.01 0.   0.   0.   0.
 0.   0.01 0.   0.   0.  ]
0.89
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  19  20  21  23
  27  3

100%|██████████| 1/1 [00:02<00:00,  2.75s/it]


[0.   0.   0.11 0.02 0.   0.01 0.84 0.   0.   0.   0.   0.   0.   0.01
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.01 0.   0.   0.   0.  ]
0.0
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  19  20  21  23
  27  39  45  49  53  63  71  84  97 100 106 119 175 184 187]
[0.   0.   0.11 0.02 0.   0.01 0.84 0.   0.   0.   0.   0.   0.   0.01
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.01 0.   0.   0.   0.  ]
0.11
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  19  20  21  23
  27  39  45  49  53  63  71  84  97 100 106 119 175 184 187]
[0.   0.   0.11 0.02 0.   0.01 0.84 0.   0.   0.   0.   0.   0.   0.01
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.01 0.   0.   0.   0.  ]
0.02
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  19  20  21  23
  27  39  45  49  53  63  71  84  97 100 106 119 175 184 187]
[0.   0.   0.11 0.02 0.   0.01 0.84 0.   0.   0.   0.   0.   0.   0.01
 0.   0.

''

In [58]:
import numpy as np

In [34]:
np.array([]).sum()

0.0

In [64]:
framework.run_results['predictions'][3][0]

array([0.1])

In [ ]:
#print(framework.test_model(randomize=False, predict_subset=1,threshold=0.7))

In [ ]:
# framework.generate_plots()

In [ ]:
import numpy as np
thresholds = np.linspace(0,1,11)
stats={"daily":{"prec":[],"rec":[]},
      "weekly":{"prec":[],"rec":[]},
      "monthly":{"prec":[],"rec":[]},
      "yearly":{"prec":[],"rec":[]},}
for threshold in thresholds:
    framework.predictor.threshold=threshold
    framework.test_model(randomize=False, predict_subset=1,save_results=False,threshold=threshold)
    for key,arr in zip(stats.keys(),framework.pred_stats):
        stats[key]["prec"].append(arr[0][1])
        stats[key]["rec"].append(arr[1][1])
        print( stats[key]["prec"], stats[key]["rec"])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
key="daily"
fig=sns.lineplot(data=pd.DataFrame(stats[key],thresholds))
fig.set_title(key)
fig.set_xlabel("threshold")
plt.savefig(key+'.png')

In [ ]:
import seaborn as sns
key="weekly"
fig=sns.lineplot(data=pd.DataFrame(stats[key],thresholds))
fig.set_title(key)
fig.set_xlabel("threshold")
plt.savefig(key+'.png')

In [ ]:
import seaborn as sns
key="monthly"
fig=sns.lineplot(data=pd.DataFrame(stats[key],thresholds))
fig.set_title(key)
fig.set_xlabel("threshold")
plt.savefig(key+'.png')

In [ ]:
import seaborn as sns
key="yearly"
fig=sns.lineplot(data=pd.DataFrame(stats[key],thresholds))
fig.set_title(key)
fig.set_xlabel("threshold")
plt.savefig(key+'.png')

In [ ]:
stats